<b> Explanations and assumptions: </b><br>
1. Collection is already opened. A function does not need to open a collection. <br>
2. Job_id index will start from 0. <br>
3. Regarding to Redis: <br>
    In oreder to support fast operations, 4 different calcualtions will be outsourced to redis.<br>
    The calculations are:<br>

    a. storing the next Job_id index.<br>

    b. storing the mapping between job_id and its company name.<br>

    c. storing the mapping between job_id and the applied candidates' emails (redis set).<br>

    d. storing the mapping between candidate's email and the comapnies it has been applied for (sorted set based on the date).<br>

4. For Company document we add attribute of total number of Jobs.



<b> Imports:

In [1]:
import pymongo
import time
import datetime
import redis 
import pandas as pd
import numpy as np
from pymongo import MongoClient

# Functions:

<b> add_company function: </b><br>
receives a dictionary (a.k.a document) of 2 keys, decides whether the company name is unique, <br>
Than adds 2 more attributes and inserts it to the collection

In [2]:
def add_company(company):
    #print collection.find_one({"company_name":company['company_name']})
    # validation: only insert if the company does not exist
    company_document = collection.find_one({"company_name":company['company_name']})
    if company_document == None:
        # insertion - once we approved the insertion, W'll add 2 more attributes and then inset the document to the collection
        company['jobs_list'] = []              # add jobs_list attribute
        company['num_of_jobs'] = 0            # add number of jobs for the company
        collection.insert_one(company)         # applying the insert function
        # Adding an indication on the system's current state for the user:
        print (company['company_name'],"was successfully added")
    else:
        print (company['company_name'],"is already exists in the system")



<b> add_job function: </b><br>
Gets a dictionary (of the job), and a string of the company's name. validates that the company exists in the DB, <br>
Then adds 2 attributes to the document. <br>
We found it better to envovled Redis in the calculations and operations of the insert section.



In [3]:
def add_job(job, company_name):
    # validation: only insert if there is an associated company
    company_document = collection.find_one({"company_name":company_name})
    if company_document == None:
        print ("Company does not exist in the DB :(")
    else:
    # insertion
        # add candidates_list attribute
        job['candidates_list'] = []
        # assign auto increment job_id attribute
        jobID = ('stud16:{}'.format('job_id'))
        job["job_id"] = jobID
                
        # add job_id to company mapping on redis;
        r.set('stud16:{}'.format(jobID), company_name)
        # update the number of positions in redis
        num_positions = int(r.get('stud16:{}'.format('job_id'))) + 1
        r.set('stud16:{}'.format('job_id'),num_positions)
                
        # update the jobs list for the company
        collection.update_many({"company_name":company_name},{"$addToSet":{"jobs_list":job}})
        collection.update_many({"company_name":company_name},{"$inc":{"num_of_jobs":1}})

<b> application function: </b><br>


In [5]:
def application(candidate, application_time, job_id):
    # find the relevant job based on job_id
    company_name = r.get('stud16:{}'.format(job_id))
    
    # validation: check that this email was not applied for this job
    # Return a boolean indicating if value is a member of set name
    if r.sismember('stud16:job{}_e'.format(job_id), candidate['email']):
        print ("Candidate already applied for this position")
        
    else:
    #insert
        #add the application time to the candidate
        candidate["application_time"] = pd.to_datetime(application_time,dayfirst=True)
        collection.update_one({"company_name":company_name, "jobs_list.job_id":job_id},{"$push":{"jobs_list.$.candidates_list":candidate}})
        # Adding to redis the candidate's emaill to the job object
        r.sadd('stud16:job{}_e'.format(job_id), candidate['email'])
        
        # Adding job application to the redis object
        # Note <time.mktime(pd.to_datetime(application_time).timetuple())> will add the time as a score
        r.zadd('stud16:{}'.format(candidate['email']),int(time.mktime(pd.to_datetime(application_time,dayfirst=True).timetuple())),company_name)

<b> show_latest_applications function: </b><br>


In [6]:
def show_latest_applications(k, candidate_email):
    # returning relevant distinct companies from candidate sorted set in redis:
    company_list =  r.zrange('stud16:{}'.format(candidate_email), 0, k, desc=True, withscores=False) 
    return company_list

<b> show_candidates function: </b><br>


In [7]:
def show_candidates(job_id, skills):
    # aggregate function - returns email from subdocuments of the relevant job_id that matches the skills criteria
    emails = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$match": { "jobs_list.job_id": str(job_id) }},
      { "$unwind": "$jobs_list.candidates_list" },
      { "$match": { "jobs_list.candidates_list.skills":{"$all":skills }} },
      { "$project": {"_id": '$jobs_list.candidates_list.email'}},
    ]
    )
    return pd.DataFrame(list(emails)).rename(columns={"_id":"Candidates' Email"})

<b> Report 1: count_jobs_by_company (date) function: </b><br>


In [8]:
def count_jobs_by_company():
    if pd.DataFrame(list(collection.find())).empty:
        return "No companies with open Jobs"
    return pd.DataFrame(list(collection.find()))[['company_name','num_of_jobs']].copy().rename(
    columns= {"company_name":"Company", "num_of_jobs": "Number of Jobs"})
    

<b> Report 2: count_candidates_by_job function: </b><br>


In [9]:
def count_candidates_by_job():
    thershold = datetime.datetime.today() - datetime.timedelta(days=30) # setting thershold (30 days) for agg. func.
    # aggregate function - returns no. of applications from subdocuments of each job
    a = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$match": { "jobs_list.candidates_list.application_time":{"$gte": thershold } }},
      { "$unwind": "$jobs_list.candidates_list" },
      { "$project": {"_id": '$jobs_list.job_id', "time": "$jobs_list.candidates_list.application_time"}},
      { "$group": {"_id": '$_id', "num_of_candidates": {'$sum': 1} }}

    ]
    )
    candidates = pd.DataFrame(list(a))
    if candidates.empty:
        return "No jobs with applications during the last 30 days"
    return candidates.rename(columns={'_id':"Job ID","num_of_candidates": "Number of Candidates"})

<b> RECOVERY function: </b><br>


In [10]:
def recovery():
    # job_id
    recover_id = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$project": {"_id": { "$max": "$jobs_list.job_id"}}},

    ]
    )
    print (recover_id)
    job_id = pd.DataFrame(list(recover_id)).max().values # get the latest job_id given in the db
    if job_id.size == 0:
        r.set('stud16:{}'.format('job_id'), 0) 
        return
    else:
        r.set('stud16:{}'.format('job_id'), int(job_id[0]) + 1)
        
    # map job_id -> company
    map_J_C = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$project": {"_id": "$jobs_list.job_id","company":"$company_name"}}

    ]
    )
    job_company_df = pd.DataFrame(list(map_J_C))
    if not job_company_df.empty:
        job_company_df['_id'] = job_company_df['_id'].apply(lambda r: "stud16:{}".format(r)) # creating the key
        job_company_dict = job_company_df.set_index('_id').T.to_dict('records')[0]
        r.mset(job_company_dict) # insert to Redis
        
    # job_id_e -> set(emails)
    email_set = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$unwind": "$jobs_list.candidates_list" },
      { "$project": {"_id": "$jobs_list.job_id","email":"$jobs_list.candidates_list.email"}}

    ]
    )
    email_set = pd.DataFrame(list(email_set))
    email_set = email_set.groupby('_id')['email'].apply(list).to_frame()
    email_set['job_id'] = email_set.index
    email_set['job_id'] = email_set['job_id'].apply(lambda r: 'stud16:job{}_e'.format(r) ) # applying key format for Redis
    email_set = email_set.set_index('job_id').T.to_dict('records')[0] # converting to dict (key,email set)
    # insert to Redis each value at a time:
    for k,v in email_set.items():
        for e in v:
            r.sadd(k,e)
            
    # candidate -> sorted set (company,date)
    company_date = collection.aggregate([
          { "$unwind": "$jobs_list" },
          { "$unwind": "$jobs_list.candidates_list" },
          { "$project": {"_id": "$jobs_list.candidates_list.email","company":"$company_name","date":"$jobs_list.candidates_list.application_time"}}

        ]
        )
    email_sorted_set = pd.DataFrame(list(company_date))
    email_sorted_set = email_sorted_set.groupby(['_id','company'],as_index=False)['date'].max() # taking the latest application sate for each company
    email_sorted_set['date'] = email_sorted_set['date'].apply(lambda r:int(time.mktime(r.timetuple())))  # convert datetime to float
    # insert to Redis:
    for k,v,score in email_sorted_set.values:
        r.zadd('stud16:{}'.format(k),score,v)


In [19]:
print (r.get('stud16:0'))

ConnectionError: Error 61 connecting to 127.0.0.1:6379. Connection refused.

<b> EXECUTION function: </b><br>
This function applies all of the functions above, with the same examples as we can find in the instructions.

In [11]:
def execute():
    recovery()
    add_company({'company_name':'TAU', 'company_description':'University'})
    add_job({'job_name':'bi developer', 'location': 'Tel Aviv',
             'skills':['python','big data','mongodb'],'status':'open',
             'publish_date':'01-02-2019'},'TAU')
    application({'candidate_name':'laura', 'email':'laura@gmail.com',
                 'linkedin':'https://www.linkedin.com/in/laura/', "skills": ['python','sql']},
                '01-02-2019 15:00:00', '0') 
    print ("Latest Applications:")
    print(show_latest_applications(10, 'laura@gmail.com'))
    print
    print ("Candidates:")
    print (show_candidates('0', ['python','sql']))
    print
    print ("Report 1 - Jobs by Company:")
    print (count_jobs_by_company())
    print
    print ("Report 2 - Candidates by Job:")
    print (count_candidates_by_job())
               

# Establishing Connections:

In [12]:
r = redis.StrictRedis(host= '127.0.0.1', port=6379)  # creating Redis connection
client = MongoClient() # creating MongoDB connection
db = client['stud16']   # Creating a database - named 'stud16'
# clean up the collection before start working
db.hm1_stud16.drop()
# creating new collection - a table in the database
collection = db.hm1_stud16

In [13]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stud16'), 'hm1_stud16')

In [14]:
collection['stud16']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stud16'), 'hm1_stud16.stud16')

In [15]:
map_J_C = collection.aggregate([
      { "$unwind": "$jobs_list" },
      { "$project": {"_id": "$jobs_list.job_id","company":"$company_name"}}

    ]
    )


In [16]:
job_company_df = pd.DataFrame(list(map_J_C))

In [17]:
job_company_df

""


In [18]:
r.flushall()

ConnectionError: Error 61 connecting to 127.0.0.1:6379. Connection refused.

##### Execution (running the script's functions):

In [ ]:
execute()

### QA section:

In [ ]:
#W'd like to add a document to our collection (==table), W'll try to add 2 collections:
#The following are part of the checks and inputs we used to validate our data model:
# company = {'company_name':'TAU', 'company_description':'University'}
# add_company(company)

# company = {'company_name':'BGU', 'company_description':'University'}
# add_company(company)

# job ={'job_name':'bi developer', 'location': 'Tel Aviv',
#     'skills':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2019'}
# add_job(job, 'TAU')

# print (collection.find_one({"company_name":'TAU'}))

# job ={'job_name':'bi developer_2', 'location': 'Tel Aviv',
#       'skills':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2019'}
# add_job(job, 'TAU')

# job ={'job_name':'bi developer', 'location': 'Tel Aviv',
#       'skills':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2019'}
# add_job(job, 'BGU')

# r.get('stud16:{}'.format(2))

# application({'candidate_name':'laura', 'email':'laura@gmail.com',
# 'linkedin':'https://www.linkedin.com/in/laura/', 'skills':['python','sql']},
# '01-05-2019 15:00:00', '1')

# application({'candidate_name':'laura', 'email':'laura@gmail.com',
# 'linkedin':'https://www.linkedin.com/in/laura/', 'skills':['python','sql']},
# '01-04-2019 15:00:00', '2')

# print (collection.find_one({"company_name":'TAU'}))

# show_latest_applications(2, 'laura@gmail.com')


In [ ]:
for res in collection.find():
    print(res)

In [ ]:
r.set('stud16:{}'.format(jobID), company_name)